In [1]:
from numba import jit
import numpy as np
from sympy import *
from datetime import datetime

Define spin-comparison and transfer matrix function on the lattice:

In [2]:
@jit(nopython=True)
def compare_spins(sp,pos1,pos2):   # set to 1 when spin on pos1 is equal to pos2, -1 otherwise
    return np.where( ((sp>>pos1) & 1) == ((sp>>pos2) & 1), 1,-1)
 
@jit(nopython=True)
def add_spin(p,size,k_max,pos,hx,hy,hz,wgt,dx,dy,spin,prime,half):      
    dx[:] = 0
    dy[:] = 0
    dz    = 0
    j=hz-1-(pos%hz) # Bitpos. at position pos.

    # calculate spin factors per spin-state s if lower x,y and z-neighbour is already in the ground-layer.
    # Before that point, all contributions remain zero due to the open boundary conditions
    if (pos>=hx): dx = wgt[0] * compare_spins(spin,j,(j+hx)%hz)
    if (pos>=hy): dy = wgt[1] * compare_spins(spin,j,(j+hy)%hz)
    if (pos>=hz): dz = wgt[2] # does not depend on spin-state

    sflip = spin^(2**j) # calculate spin-config with flipped lower z-neighbor spin on pos j
    if ( j==(hz-1) ): sflip = (2**hz)-1 - sflip # highest spin, flip all other spins to use Z(2) Symm.
    
    # update series expansion via recursion-step:
    for k in range(k_max, 0, -1): # k stays above 0 and the 0th-order term will not be updated here!
        pnew = (p[k  ,:]            + p[k-2,:]*(dx*dy+dx*dz+dy*dz))%prime
        term = (p[k-1,:]*(dx+dy+dz) + p[k-3,:]*(dx*dy*dz))%prime
        preg = (pnew - term)%prime
        pnew = (pnew + term)%prime
        p[k,:] = ((pnew + preg[sflip]) * half)%prime

In [7]:
# Set Parameter for Lattice
k_max  = 24 # maximum order for series expansion
layers = 12 # number of layers of hz-spins
hx,hy,hz = (17,21,22) #
nconf = 2**(hz-1) # half of required spin-configs due to use of Z(2) Symmetrie
dx    = np.zeros(nconf, dtype=int)
dy    = np.zeros(nconf, dtype=int)
spin  = np.arange(nconf, dtype=int)
part1 = np.zeros([8,k_max+3], dtype=object)
part2 = np.zeros([8,k_max+3], dtype=object)

# define weight factors to eliminate unphysical loops
weight = np.array([[1,1,1],[-1,1,1],[1,-1,1],[1,1,-1],[-1,-1,1],[1,-1,-1],[-1,1,-1],[-1,-1,-1]])
# define parameter to use Chinese Remainder Theorem to limit the size of integers:
#primes = [715225739, 715225741, 715225747] # disjoint primes for CRT (Chinese Remainder Theorem)
#halfes = [357612870, 357612871, 357612874]
#bases  = [297271162282471677373010151, 30489349931781261206660785, 38111687478670059350160518]
#limit  = 365872199692922997929831453
primes = [715225739, 715225741, 715225747, 715225787]
halfes = [357612870, 357612871, 357612874, 357612894]
bases  = [249755654989615123966013739719281184, 154069565921814367522325857974438752, 223110508713543954068849992613458676, 158107966275402583017099624940656922]
limit  = 261681231966792009524763071749278511 


# define partition functions
t=Symbol('t')
z1=Symbol('z1')
z2=Symbol('z2')
F=Symbol('F')
F = 0

print('Lattice: (',hx,',',hy,',',hz,')')
for w in range(0,1): # loop over weigths to be able to cancel unphysical FS-loops
 
    timestamp = datetime.now().strftime("%H:%M:%S")
    print('w=',w, 'at', timestamp)
    for m in range(0,4): # loop over primes to avoid overflow using the CRT (Chinese Remainder Theorem)
        # init partition function (with 2 extra rows -1 and -2 to simplify recursion steps)
        p = np.zeros((k_max+3, nconf), dtype=int) # reset partition function
        p[0,:] = 1 # init partition function and set trivial zeroth order to 1
    
        #  Now build up lattice spin by spin, layer by layer:
        for pos in range (0, layers*hz):
            timestamp = datetime.now().strftime("%H:%M:%S")
            print('prime=',m,'pos=',pos, 'at', timestamp)
            add_spin(p,nconf,k_max,pos,hx,hy,hz,weight[w],dx,dy,spin,primes[m],halfes[m])
        # sum into 1st partition function using CRT-bases:
        part1[w,:] = (part1[w,:] + bases[m]*(np.sum(p, axis=1)%primes[m]))%limit 
        
        # add one additional spin to eliminate surface effects later:
        add_spin(p,nconf,k_max,layers*hz,hx,hy,hz,weight[w],dx,dy,spin,primes[m],halfes[m])
        # sum into 2nd partition function using CRT-bases:
        part2[w,:] = (part2[w,:] + bases[m]*(np.sum(p, axis=1)%primes[m]))%limit 
        
    # insert the correct signs:
    part1[w,:] = np.where(part1[w,:]>limit/2,part1[w,:]-limit,part1[w,:])
    part2[w,:] = np.where(part2[w,:]>limit/2,part2[w,:]-limit,part2[w,:])
    
    # build series for partition functions:
    z1 = sum(part1[w,i]*t**i for i in range(0,k_max+1))
    z2 = sum(part2[w,i]*t**i for i in range(0,k_max+1))
    F = F + series(log(z2)-log(z1),x=t,x0=0,n=25) # Free energy without surface effects
F

Lattice: ( 17 , 21 , 22 )
w= 0 at 19:43:21
prime= 0 pos= 0 at 19:43:21
prime= 0 pos= 1 at 19:43:24
prime= 0 pos= 2 at 19:43:26
prime= 0 pos= 3 at 19:43:28
prime= 0 pos= 4 at 19:43:29
prime= 0 pos= 5 at 19:43:31
prime= 0 pos= 6 at 19:43:33
prime= 0 pos= 7 at 19:43:35
prime= 0 pos= 8 at 19:43:37
prime= 0 pos= 9 at 19:43:39
prime= 0 pos= 10 at 19:43:40
prime= 0 pos= 11 at 19:43:42
prime= 0 pos= 12 at 19:43:44
prime= 0 pos= 13 at 19:43:46
prime= 0 pos= 14 at 19:43:48
prime= 0 pos= 15 at 19:43:50
prime= 0 pos= 16 at 19:43:52
prime= 0 pos= 17 at 19:43:53
prime= 0 pos= 18 at 19:43:55
prime= 0 pos= 19 at 19:43:57
prime= 0 pos= 20 at 19:43:59
prime= 0 pos= 21 at 19:44:01
prime= 0 pos= 22 at 19:44:03
prime= 0 pos= 23 at 19:44:05
prime= 0 pos= 24 at 19:44:07
prime= 0 pos= 25 at 19:44:10
prime= 0 pos= 26 at 19:44:12
prime= 0 pos= 27 at 19:44:15
prime= 0 pos= 28 at 19:44:17
prime= 0 pos= 29 at 19:44:20
prime= 0 pos= 30 at 19:44:22
prime= 0 pos= 31 at 19:44:25
prime= 0 pos= 32 at 19:44:27
prime= 0 p

3*t**4/8 + 11*t**6/4 + 375*t**8/16 + 7*t**9 + 1053*t**10/4 + 1365*t**11/4 + 7439*t**12/2 + 35355*t**13/4 + 115491*t**14/2 + 719545*t**15/4 + 29581479*t**16/32 + 13161381*t**17/4 + 179993357*t**18/12 + 230216075*t**19/4 + 4919950979*t**20/20 + 3952145191*t**21/4 + 32481238235*t**22/8 + 67006271467*t**23/4 + 267877856235*t**24/4 + O(t**25)

In [8]:
part1

array([[2097152, 0, 0, 0, 1409286144, 0, 9714008064, 0, 550410125312,
        24373100544, 7336080965632, 1116779577344, 191056045932544,
        43456583958528, 3251626924048384, 1377314306785280,
        65117384557461504, 38498202382499840, 1170920436647591936,
        990508404189429760, 21558783772305915904, 23717273715632766976,
        386495853005914505216, 539193249975767138304,
        6915733991387564081152, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
      

In [9]:
part2

array([[2097152, 0, 0, 0, 1415577600, 0, 9760145408, 0, 555040636928,
        24490541056, 7400782299136, 1122504802304, 193255666876416,
        43757265223680, 3291661165658112, 1388629939191808,
        66037632628424704, 38870846718083072, 1188703547875655680,
        1001486336412090368, 21914683468085723136, 24012741025464320000,
        393287466381519880192, 546639938329415516160,
        7044675980321844363264, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
     

In [92]:
f1

log(4096) + 402*t**4 + 2782*t**6 + 101921*t**8 + 1166420*t**10 + 11437448*t**12 + 102251234*t**14 - 2976371871*t**16/2 - 153300876926*t**18/3 - 4454280735043*t**20/5 - 12145546501076*t**22 + O(t**24)

In [81]:
Z1

Matrix([
[               16703792423750025216*t**24 + 983712715251208192*t**22 + 57763413272637440*t**20 + 3187057341005824*t**18 + 185150526455808*t**16 + 8875501228032*t**14 + 549997875200*t**12 + 18724298752*t**10 + 1498152960*t**8 + 21798912*t**6 + 3330048*t**4 + z1 + 4096],
[          33407586310703357952*t**24 + 1967425295206273024*t**22 + 115526774653399040*t**20 + 6374132158668800*t**18 + 370299600437248*t**16 + 17750718468096*t**14 + 1100180574208*t**12 + 37441249280*t**10 + 2995019776*t**8 + 44589056*t**6 + 6623232*t**4 + z1 + 12288],
[        64653168342177103872*t**24 + 3847523504841490432*t**22 + 227856915178233856*t**20 + 12643480402411520*t**18 + 737756462481408*t**16 + 35437427326976*t**14 + 2199598800896*t**12 + 74882498560*t**10 + 5990039552*t**8 + 89178112*t**6 + 13246464*t**4 + z1 + 24576],
[     95898750479070167040*t**24 + 5727621872956211200*t**22 + 340187023765790720*t**20 + 18912831081267200*t**18 + 1105213852598272*t**16 + 53123749871616*t**14 + 3299078578176*

In [75]:
M = M.col_insert(1,Matrix(z1))

In [76]:
M

Matrix([
[               32768,                32768],
[                   0,                    0],
[                   0,                    0],
[                   0,                    0],
[            13172736,             13172736],
[                   0,                    0],
[            91160576,             91160576],
[                   0,                    0],
[          5987467264,           5987467264],
[                   0,                    0],
[         74867802112,          74867802112],
[                   0,                    0],
[       2198959554560,        2198959554560],
[                   0,                    0],
[      35372645089280,       35372645089280],
[                   0,                    0],
[     734914780233728,      734914780233728],
[                   0,                    0],
[   12538701357711360,    12538701357711360],
[                   0,                    0],
[  224660217175113728,   224660217175113728],
[                   0,   

In [30]:
F=series(log(Z2)-log(Z1),x=t,x0=0,n=25)

In [8]:
z1_1

NameError: name 'z1_1' is not defined

In [28]:
symbols('a0:3')

(a0, a1, a2)

In [38]:
z1_0

t

In [36]:
M

Matrix([
[0],
[1],
[2],
[3],
[4],
[5],
[6],
[7]])